In [1]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [3]:
!pip install sklearn-crfsuite

     |████████████████████████████████| 747kB 4.0MB/s 


In [4]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics


# 파일 경로
file_path = "/gdrive/My Drive/colab/Sequence Labeling/week6/spacing_data.txt"

# "spacing_data.txt" 파일을 읽고 lines에 읽은 데이터를 저장
with open(file_path, "r", encoding="utf8") as inFile:
    lines = inFile.readlines()

# 데이터를 음절로 이루어진 문장과 정답 값으로 나누어 저장
datas = []
for line in lines:
    pieces = line.strip().split("\t")
    eumjeol_sequence, label = pieces[0].split(), pieces[1].split()
    datas.append((eumjeol_sequence, label))
    
number_of_train_datas = int(len(datas)*0.9)

train_datas = datas[:number_of_train_datas]
test_datas = datas[number_of_train_datas:]
          
print("train_datas 개수 : " + str(len(train_datas)))          
print("test_datas 개수 : " + str(len(test_datas)))

for data in train_datas[:5]:
  print(data)

train_datas 개수 : 900
test_datas 개수 : 100
(['약', '속', '장', '소', '인', '신', '라', '호', '텔', '커', '피', '숍', '에', '재', '옥', '이', '먼', '저', '와', '기', '다', '리', '고', '있', '었', '다', '.'], ['B', 'I', 'B', 'I', 'I', 'B', 'I', 'I', 'I', 'B', 'I', 'I', 'I', 'B', 'I', 'I', 'B', 'I', 'B', 'B', 'I', 'I', 'I', 'B', 'I', 'I', 'I'])
(['"', '야', '!', '나', '이', '든', '처', '녀', '가', '옷', '이', '라', '도', '좀', '화', '사', '한', '색', '으', '로', '입', '고', '다', '녀', '라', '.', '회', '색', '에', '다', '검', '정', '바', '바', '리', '가', '뭐', '니', '?', '"'], ['B', 'I', 'I', 'B', 'I', 'B', 'B', 'I', 'I', 'B', 'I', 'I', 'I', 'B', 'B', 'I', 'I', 'B', 'I', 'I', 'B', 'I', 'B', 'I', 'I', 'I', 'B', 'I', 'I', 'I', 'B', 'I', 'B', 'I', 'I', 'I', 'B', 'I', 'I', 'I'])
(['재', '옥', '은', '전', '에', '없', '이', '정', '현', '의', '옷', '차', '림', '을', '탓', '하', '였', '다', '.'], ['B', 'I', 'I', 'B', 'I', 'B', 'I', 'B', 'I', 'I', 'B', 'I', 'I', 'I', 'B', 'I', 'I', 'I', 'I'])
(['"', '이', '게', '어', '때', '서', '?', '갑', '자', '기', '옷', '얘', '긴', '왜', '하', '니', '?

In [14]:
def sent2feature(eumjeol_sequence):
  features = []
  sequence_length = len(eumjeol_sequence)
  for index in range(sequence_length):
      if index == 0:
          feature = {"WORD":eumjeol_sequence[index], "F_1":'^', "F_2":eumjeol_sequence[index+1]}
      
      elif index == sequence_length - 1:
          feature = {"WORD":eumjeol_sequence[index], "F_1":eumjeol_sequence[index-1], "F_2":'$'}

      else: feature = {"WORD":eumjeol_sequence[index], "F_1":eumjeol_sequence[index-1], "F_2":eumjeol_sequence[index+1]}

      features.append(feature)
  return features
  
train_x, train_y = [], []
for eumjeol_sequence, label in train_datas:
    train_x.append(sent2feature(eumjeol_sequence))
    train_y.append(label)

test_x, test_y = [], []
for eumjeol_sequence, label in test_datas:
    test_x.append(sent2feature(eumjeol_sequence))
    test_y.append(label)

print(train_x[:5],'\n\n', train_y[:5])

[[{'WORD': '약', 'F_1': '^', 'F_2': '속'}, {'WORD': '속', 'F_1': '약', 'F_2': '장'}, {'WORD': '장', 'F_1': '속', 'F_2': '소'}, {'WORD': '소', 'F_1': '장', 'F_2': '인'}, {'WORD': '인', 'F_1': '소', 'F_2': '신'}, {'WORD': '신', 'F_1': '인', 'F_2': '라'}, {'WORD': '라', 'F_1': '신', 'F_2': '호'}, {'WORD': '호', 'F_1': '라', 'F_2': '텔'}, {'WORD': '텔', 'F_1': '호', 'F_2': '커'}, {'WORD': '커', 'F_1': '텔', 'F_2': '피'}, {'WORD': '피', 'F_1': '커', 'F_2': '숍'}, {'WORD': '숍', 'F_1': '피', 'F_2': '에'}, {'WORD': '에', 'F_1': '숍', 'F_2': '재'}, {'WORD': '재', 'F_1': '에', 'F_2': '옥'}, {'WORD': '옥', 'F_1': '재', 'F_2': '이'}, {'WORD': '이', 'F_1': '옥', 'F_2': '먼'}, {'WORD': '먼', 'F_1': '이', 'F_2': '저'}, {'WORD': '저', 'F_1': '먼', 'F_2': '와'}, {'WORD': '와', 'F_1': '저', 'F_2': '기'}, {'WORD': '기', 'F_1': '와', 'F_2': '다'}, {'WORD': '다', 'F_1': '기', 'F_2': '리'}, {'WORD': '리', 'F_1': '다', 'F_2': '고'}, {'WORD': '고', 'F_1': '리', 'F_2': '있'}, {'WORD': '있', 'F_1': '고', 'F_2': '었'}, {'WORD': '었', 'F_1': '있', 'F_2': '다'}, {'WORD': '다', 'F_1': '었

In [12]:
crf = sklearn_crfsuite.CRF(algorithm='lbfgs')
crf.fit(train_x, train_y)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=None,
    averaging=None, c=None, c1=None, c2=None, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=None,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [16]:
def show_predict_result(test_datas, predict):
  for index_1 in range(len(test_datas)):
      eumjeol_sequence, correct_labels = test_datas[index_1]
      predict_labels = predict[index_1]
     
      correct_sentence, predict_sentence = "", ""
      for index_2 in range(len(eumjeol_sequence)):
          if(index_2 == 0):
              correct_sentence += eumjeol_sequence[index_2]
              predict_sentence += eumjeol_sequence[index_2]
              continue

          if(correct_labels[index_2] == "B"):
              correct_sentence += " "
          correct_sentence += eumjeol_sequence[index_2]

          if (predict_labels[index_2] == "B"):
              predict_sentence += " "
          predict_sentence += eumjeol_sequence[index_2]

      print("정답 문장 : " + correct_sentence)
      print("출력 문장 : " + predict_sentence)
      print()

predict = crf.predict(test_x)

print("Accuracy score : " + str(metrics.flat_accuracy_score(test_y, predict)))
print("\n10개의 데이터에 대한 모델 출력과 실제 정답 비교\n")

show_predict_result(test_datas[:10], predict[:10])

Accuracy score : 0.8994658527279664

10개의 데이터에 대한 모델 출력과 실제 정답 비교

정답 문장 : 1914- 18년의 전쟁은 인류를 통합시킨 최초의 공통분모였다.
출력 문장 : 19 14- 18년의 전쟁은 인류를 통합시킨 최초의 공통 분모였다.

정답 문장 : 하지만 이 전쟁은 죽음을 통해 인류를 통합시켰다.
출력 문장 : 하지만이 전쟁은 죽음을 통해 인류를 통합시켰다.

정답 문장 : 사라예보에서 한 세르비아인이 쏜 총 한발이 합스부르크가의 계승자를 죽였다.
출력 문장 : 사라 예보에서 한세르 비아인이 쏜총한 발이 합스부르크가의 계승자를 죽였다.

정답 문장 : 이 암살행위는 국지적인 민족주의들과 세계적인 제국주의들이 충돌하는 분쟁지역에서 저질러졌다.
출력 문장 : 이암 살행 위는 국지적인 민족주의 들과 세계적인 제국주의 들이 충돌하는 분쟁 지역에서 저질러졌다.

정답 문장 : 오토만제국의 점진적인 해체는 민족주의의 독기를 발산하는 동시에 오스트리아, 헝가리와 독일, 영국, 프랑스의 탐욕을 자극했다.
출력 문장 : 오토만 제국의 점진적인해체는 민족주의의 독기를 발산하는 동시에 오스 트리아, 헝가리와 독일, 영국, 프랑스의 탐욕을 자극했다.

정답 문장 : 이렇게 해서 발칸 반도의 한 외진 장소에서 벌어진 국지적인 테러 행위는 일련의 긴박한 반응을 불러 일으키면서 전 유럽에 영향을 미쳤을 뿐만 아니라 이번에는 아시아와 아프리카 식민지들, 일본, 그리고 이어서 미국과 멕시코까지 끌어들였다.
출력 문장 : 이렇게 해서 발칸 반도의 한외진 장소에서 벌어 진국지적인테 러행 위는 일련의 긴박한 반응을 불러일으키면서 전유럽에 영향을 미쳤을 뿐만 아니라이 번에는 아시아와 아프리 카식민지들, 일본, 그리고 이어서 미국과 멕시코까지 끌어들였다.

정답 문장 : 전쟁의 물결이 지구상의 모든 대양으로 밀려드는 동안 캐나다인들과 미국인들, 오스트레일리아인들, 세네갈인들, 알제리인들, 모로코인들, 안남(安南)인들은 연합군 깃발을 휘날리며 유럽전선에